In [4]:
cd Desktop/Summer\ research/

[Errno 2] No such file or directory: 'Desktop/Summer research/'
/Users/wuyifan/Desktop/Summer research


In [45]:
import numpy as np
import math
import igl
import meshplot as mp
import wildmeshing as wm
import polyfempy as pf
import scipy.sparse as sp
import random
import os

name = "cuboid"
path = "fem/data/"+name+"_l"+".obj"
out = "fem/out/out_"+name+"_260"+".mesh"

def add_axis(plot, plot1):
    plot.add_lines(np.array([[0,0,0],[50,0,0],[50,0,0]]), np.array([[50,0,0],[48,2,0],[48,-2,0]]), shading={"line_color": "blue"})
    plot.add_lines(np.array([[0,0,0],[0,50,0],[0,50,0]]), np.array([[0,50,0],[-2,48,0],[2,48,0]]), shading={"line_color": "red"})
    plot.add_lines(np.array([[0,0,0],[0,0,50],[0,0,50]]), np.array([[0,0,50],[0,2,48],[0,-2,48]]), shading={"line_color": "yellow"})
    plot1.add_lines(np.array([[0,0,0],[50,0,0],[50,0,0]]), np.array([[50,0,0],[48,2,0],[48,-2,0]]), shading={"line_color": "blue"})
    plot1.add_lines(np.array([[0,0,0],[0,50,0],[0,50,0]]), np.array([[0,50,0],[-2,48,0],[2,48,0]]), shading={"line_color": "red"})
    plot1.add_lines(np.array([[0,0,0],[0,0,50],[0,0,50]]), np.array([[0,0,50],[0,2,48],[0,-2,48]]), shading={"line_color": "yellow"})
    
def add_force(plot, plot1, mag, force_ax, ex):
    mag = np.array(mag)
    dir1 = -mag/4
    dir2 = -mag/4
    arrow_ax = random.randint(0,2)
    
    minn = np.min(ex, axis=0)
    maxx = np.max(ex, axis=0)
    center = (minn+maxx)/2
    
    while arrow_ax == force_ax:
        arrow_ax = random.randint(0,2)
    rand = np.array([0,0,0]) # random.randint(0,1),random.randint(0,1),random.randint(0,1)
    rand[arrow_ax] = 2
    
    dir1 += rand
    dir2 += -rand
    ex = ps[s[:,0] == force, :]
    mag1 = mag*2
    
    e1 = np.vstack((np.vstack((center+mag1, center+mag1)), center+mag1))
    e2 = np.vstack((np.vstack((center-mag1, center+mag1+dir1)), center+mag1+dir2))
    plot.add_lines(e1, e2, shading={"line_color": "black", "line_width": 5.0})
    
    #plot.add_points(np.array([center]), shading={"point_color": "white", "point_size": 10})
    
    plot1.add_lines(e1, e2, shading={"line_color": "black", "line_width": 5.0})
    
    """
    e11 = np.vstack((np.vstack((ex[0:5]+mag1, ex[0:5]+mag1)), ex[0:5]+mag1))
    e12 = np.vstack((np.vstack((ex[0:5]-mag1, ex[0:5]+mag1+dir1)), ex[0:5]+mag1+dir2))
    plot.add_lines(e11, e12, shading={"line_color": "black"})
    e13 = np.vstack((np.vstack((ex[50:55]+mag1, ex[50:55]+mag1)), ex[50:55]+mag1))
    e14 = np.vstack((np.vstack((ex[50:55]-mag1, ex[50:55]+mag1+dir1)), ex[50:55]+mag1+dir2))
    plot.add_lines(e13, e14, shading={"line_color": "black"})
    """
    
def add_fix(plot, plot1):
    plot.add_lines(np.array([[-40, -10, -16],[-40, -10, 16],[40, -10, -16],[-40, -10, -16]]), np.array([[40, -10, -16],[40, -10, 16],[40, -10, 16],[-40, -10, 16]]), shading={"line_color": "brown", "line_width": 2.5})
    #plot.add_points(e2, shading={"point_color": "white", "point_size": 4})
    plot1.add_lines(np.array([[-40, -10, -16],[-40, -10, 16],[40, -10, -16],[-40, -10, -16]]), np.array([[40, -10, -16],[40, -10, 16],[40, -10, 16],[-40, -10, 16]]), shading={"line_color": "brown", "line_width": 2.5})
    
    
def collect_result(path_y):
    #print(len(files_y))

    y_f = open(path_y)
    with y_f as file:
        y = np.array([[float(digit) for digit in line.split()] for line in file])
    return y

def plot_error(p_uni, t_uni, stress_result, mises_u):
    
    
    stress_result = stress_result.reshape(260,1)
    print(stress_result.shape)
    print(mises_u.shape)
    dif = mises_u - stress_result
    print(dif.shape)
    plot = mp.plot(p_uni, t_uni, dif, return_plot=True)
    plot.save("fem/plots/error_" + str(loss) + "_" + str(force) + str(mag))
    

In [30]:
force = 6
mag = [5,10,3]
loss = 2.2037
loss = format(loss, '.4f')

path_y = "fem/test-force+/y_" + str(loss) + " 2 " + str(force) + str(mag) + ".txt"
y = collect_result(path_y)
    
solver = pf.Solver()
solver.load_mesh_from_path(out, vismesh_rel_area=1e-3)

v, f = igl.read_triangle_mesh(out)   
minn = np.min(v, axis=0)
maxx = np.max(v, axis=0)

face_idx = [[0,minn[0],"left"], [1,minn[1], "bottom"], [0, maxx[0], "right"], [1, maxx[1], "top"], [2, minn[2], "back"], [2, maxx[2], "front"]]
force_ax = face_idx[force-1][0]

ps, ts, s = solver.get_boundary_sidesets()

settings = pf.Settings()
problem = pf.Problem()
settings.set_pde(pf.PDEs.LinearElasticity)
settings.set_material_params("E", 10000)
settings.set_material_params("nu", 0.35)

fix = 2
    
# set the displacement value for the sideset id
problem.set_displacement(fix, [0, 0, 0])
problem.set_force(force, mag)
settings.set_problem(problem)
solver.settings(settings)
solver.solve()

p, tri, disp = solver.get_sampled_solution()

m = np.linalg.norm(disp, axis=1)

p_uni, indices, inverse = np.unique(p, return_index=True, return_inverse=True, axis=0)
t_uni = np.array([inverse[tri[:, 0]], inverse[tri[:, 1]], inverse[tri[:, 2]], inverse[tri[:, 3]]]).transpose()
d_uni = disp[indices, :]
m_uni = m[indices]

mises, stress_ten = solver.get_sampled_mises_avg()
mises_u = mises[indices, :]
    
stress_result = y[:,0]
disp_result = np.array([x[1:] for x in y])
disp_result /= 1000


plot = mp.plot(p_uni+d_uni, t_uni, mises_u, return_plot=True)
# plot direction and magnitude of the force
ex = ps[s[:,0] == force, :]
    
plot1 = mp.plot(p_uni+disp_result, t_uni, stress_result, return_plot=True)
add_force(plot, plot1, mag, force_ax, ex)
add_fix(plot, plot1)
# plot x,y,z-axis
add_axis(plot,plot1)
plot.save("fem/plots/gr_" + str(force) + str(mag))
plot1.save("fem/plots/result_" + str(loss) + "_" + str(force) + str(mag))

plot_error(p_uni, t_uni, stress_result, mises_u)
#print(mises_u-stress_result)
"""
mises_u.reshape(260,1)
stress_result.reshape(260,1)
po = np.zeros([260,1])
for i in range(260):
    po[i] = stress_result[i]/mises_u[i]
s = np.sum(po)
print(s/260)

d_uni.reshape(260,3)
disp_result.reshape(260,3)
po = np.zeros([260,3])
count = 0
for i in range(260):
    for j in range(3):
        if not d_uni[i][j] == 0:
            po[i][j] = disp_result[i][j] / d_uni[i][j]
            count += 1
s = np.sum(po)
print(s/count)
"""
#print(po.shape)

[2020-07-02 18:12:43.578] [polyfem] [info] Loading mesh...
[2020-07-02 18:12:43.578] [geogram] [info] Loading file fem/out/out_cuboid_260.mesh...
[2020-07-02 18:12:43.584] [geogram] [info] (FP64) nb_v:260 nb_e:0 nb_f:386 nb_b:0 tri:1 dim:3
[2020-07-02 18:12:43.584] [geogram] [info]  nb_tets:872
[2020-07-02 18:12:43.584] [geogram] [info] Attributes on vertices: point[3]
[2020-07-02 18:12:43.595] [polyfem] [info] mesh bb min [-30, -10, -6], max [30, 10, 6]
[2020-07-02 18:12:43.595] [polyfem] [info]  took 0.0172295s
[2020-07-02 18:12:43.608] [polyfem] [info] simplex_count: 	872
[2020-07-02 18:12:43.608] [polyfem] [info] regular_count: 	0
[2020-07-02 18:12:43.608] [polyfem] [info] regular_boundary_count: 	0
[2020-07-02 18:12:43.608] [polyfem] [info] simple_singular_count: 	0
[2020-07-02 18:12:43.608] [polyfem] [info] multi_singular_count: 	0
[2020-07-02 18:12:43.608] [polyfem] [info] boundary_count: 	0
[2020-07-02 18:12:43.608] [polyfem] [info] multi_singular_boundary_count: 	0
[2020-07-02

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0158996…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0155487…

Plot saved to file fem/plots/gr_6[5, 10, 3].html.
Plot saved to file fem/plots/result_2.2037_6[5, 10, 3].html.
(260, 1)
(260, 1)
(260, 1)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Plot saved to file fem/plots-simple/error_2.2037_6[5, 10, 3].html.


'\nmises_u.reshape(260,1)\nstress_result.reshape(260,1)\npo = np.zeros([260,1])\nfor i in range(260):\n    po[i] = stress_result[i]/mises_u[i]\ns = np.sum(po)\nprint(s/260)\n\nd_uni.reshape(260,3)\ndisp_result.reshape(260,3)\npo = np.zeros([260,3])\ncount = 0\nfor i in range(260):\n    for j in range(3):\n        if not d_uni[i][j] == 0:\n            po[i][j] = disp_result[i][j] / d_uni[i][j]\n            count += 1\ns = np.sum(po)\nprint(s/count)\n'